In [1]:
import os
import torch
import json

from bcnf.utils import get_dir, load_config
from bcnf.train import Trainer
from bcnf import CondRealNVP

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
MODEL_NAME = 'trajectory_LSTM_large'

In [3]:
config_path = os.path.join(get_dir("configs", "runs"), f"{MODEL_NAME}.yaml")
config = load_config(config_path)

In [4]:
model = CondRealNVP.from_config(config).to(device)

print(f'{model.n_params:,}')

51,558,407


In [5]:
trainer = Trainer(
    config={k.lower(): v for k, v in config.to_dict().items()},
    project_name="bcnf-test",
    parameter_index_mapping=model.parameter_index_mapping,
    verbose=True,
)

Using dtype: torch.float32
Loading data from /home/computerman/Desktop/IGNNS/IGNNS-final-project/data/bcnf-data/fixed_data_render_2s_15FPS/train...


Loading data from directory: 100%|██████████| 4/4 [00:42<00:00, 10.55s/it, file=fixed_data_render_2s_15FPS_4.pkl]


Using trajectories data for training. Shapes:
X shape: torch.Size([4000, 30, 3])
y shape: torch.Size([4000, 19])


In [6]:
model = trainer.train(model)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ichich111222333 (balisticcnf). Use `wandb login --relogin` to force relogin


Train: -21.4394 - Val: -15.9474 (avg: -15.2819, min: -15.2419) | lr: 2.00e-04 - Patience: 75/75 - z: (0.0189 ± 0.3313) ± (1.0692 ± 0.1876):   1%|          | 282/50000 [04:23<12:54:11,  1.07it/s] 
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


distance_to_last_best_val_loss_fold_-1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▃▄▄▅▆▆▇█
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
lr_fold_-1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
time_fold_-1,▁▁▁▁▁▁▁▁▁▁▁▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅██████████
train_loss_fold_-1,█▇▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
val_loss_fold_-1,█▇▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
z_mean_mean_fold_-1,▅▄▃▃▄▆▂▄▆▅▃▂▄▆▄▅▄▄▄▇▄▅▃▅▆▄▄█▂▃▄▅▄▅▁▃▆▃▅▅
z_mean_std_fold_-1,▄▃▂▃▂▂▄▃▃▃▁▄▃▃▄▅▄▂▆▄▄▄▁▃▅▄▄▇▄▃▄▄▄▇▄▇▅▅▂█
z_std_mean_fold_-1,▁▃▁▂▂▃▁▄▂▃▄▃▃▂▂▂▂▂▄▃▃▅▄▄▇▅▅▅▅▄▄▇▅██▇▇▇▆▇
z_std_std_fold_-1,▄▃▁▁▃▄▂▅▄▄▆▃▂▃▃▄▂▁▄▃▂▄▃▃█▃▃▄▃▃▄█▆▆▆▅▅▅▆▃
distance_to_last_best_val_loss_fold_-1,75


In [7]:
torch.save(model.state_dict(), os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), f"state_dict.pt"))

with open(os.path.join(get_dir('models', 'bcnf-models', MODEL_NAME, create=True), 'config.json'), 'w') as f:
    json.dump({'config_path': config_path}, f)

print(f"Model saved to {get_dir('models', 'bcnf-models', MODEL_NAME)}")

Model saved to /home/computerman/Desktop/IGNNS/IGNNS-final-project/models/bcnf-models/trajectory_LSTM_large
